In [3]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score
from sklearn import svm
from pandas import ExcelWriter
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np
import re


def xy_sets(line):
    X=None
    y=None
    indexes = map(int, re.findall('\d+', line))
    aux = False
    for i in indexes:
        if aux is False:
            X = scaled[i, :]
            y = target[i]
            aux = True
            continue
        X = np.vstack((X, scaled[i, :]))
        y = np.hstack((y, target[i]))
        
    return X, y, indexes


multiplier = ['0,05', '0,01', '0,001']
kers = ['linear', 'poly', 'rbf']
dataset_name = ['57EDG', '54BND', 'combined']

for name in dataset_name:
    writer = ExcelWriter(name+'_svm.xlsx', engine='xlsxwriter')
    df = DataFrame()
    temp = DataFrame()
    for m in multiplier:
        for ker in kers:
            # load dataset
            dataset = read_csv(name+'_'+m+'.csv', header=0, index_col=(0,1))
            values = dataset.values

            # split data set into data(input) and target(output)
            data = values[:, 1:] 
            target = values[:, 0]

            # normalize features
            scaled = normalize(data[:, :3], norm='max', axis=0)
            scaled = np.concatenate((scaled, data[:, :4]), axis = 1)

            #create the SVM
            clf = svm.SVC(gamma='auto', kernel=ker)

            scores = np.zeros(20)
            cont = 0
            sensitivity = 0  # true positives rate
            specificity = 0  # true negatives rate
            y_test_full = np.array([])
            y_cont_full = np.array([])
            
            file = open(name+'_kf.txt', 'r')
            while cont < 20:

                train_line = file.readline()
                test_line = file.readline()

                X_train, y_train, train_indexes = xy_sets(train_line)
                X_test, y_test, test_indexes = xy_sets(test_line)

                clf.fit(X_train, y_train)
                scores[cont] = clf.score(X_test, y_test)
                y_pred = clf.predict(X_test)
                
                y_test_full = np.hstack((y_test_full, y_test))
                y_cont_full = np.hstack((y_cont_full, y_pred))
                
                cont += 1
                
            file.close()
            
            print(confusion_matrix(y_test_full, y_cont_full))
            
            acc = accuracy_score(y_test_full, y_cont_full)
            recall = recall_score(y_test_full, y_cont_full, average=None)
            sns = recall[0]
            spf = recall[1]
            
            col = np.copy(scores)
            col = np.hstack((col, acc))
            col = np.hstack((col, sns))
            col = np.hstack((col, spf))
            # col = np.hstack((col, std))

            df.insert(0, ker, col, True)
            last = df.index[-1]
            df = df.rename(index={last: 'specificity'})
            last = df.index[-2]
            df = df.rename(index={last: 'sensibility'})
            last = df.index[-3]
            df = df.rename(index={last: 'accuracy'})
            print(scores.mean())
            print("Accuracy: %0.2f" % (acc))
            print("Sensitivity: %0.2f" % (sns))
            print("Specificity: %0.2f" % (spf))
            
    df.to_excel(writer, sheet_name=name)
    writer.save()
    writer.close()
    

[[37  0]
 [13  7]]
0.775
Accuracy: 0.77
Sensitivity: 1.00
Specificity: 0.35
[[37  0]
 [20  0]]
0.65
Accuracy: 0.65
Sensitivity: 1.00
Specificity: 0.00
[[37  0]
 [19  1]]
0.6666666666666667
Accuracy: 0.67
Sensitivity: 1.00
Specificity: 0.05
[[36  1]
 [ 5 15]]
0.9
Accuracy: 0.89
Sensitivity: 0.97
Specificity: 0.75
[[37  0]
 [ 9 11]]
0.8416666666666666
Accuracy: 0.84
Sensitivity: 1.00
Specificity: 0.55
[[37  0]
 [ 6 14]]
0.9
Accuracy: 0.89
Sensitivity: 1.00
Specificity: 0.70
[[36  1]
 [ 4 16]]
0.9166666666666666
Accuracy: 0.91
Sensitivity: 0.97
Specificity: 0.80
[[37  0]
 [ 8 12]]
0.8666666666666666
Accuracy: 0.86
Sensitivity: 1.00
Specificity: 0.60
[[37  0]
 [ 4 16]]
0.9333333333333332
Accuracy: 0.93
Sensitivity: 1.00
Specificity: 0.80
[[17 11]
 [25  1]]
0.3083333333333333
Accuracy: 0.33
Sensitivity: 0.61
Specificity: 0.04
[[24  4]
 [26  0]]
0.4333333333333333
Accuracy: 0.44
Sensitivity: 0.86
Specificity: 0.00
[[23  5]
 [26  0]]
0.4083333333333333
Accuracy: 0.43
Sensitivity: 0.82
Specifi